In [42]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
%matplotlib inline

import talib.abstract as ta

import warnings
warnings.filterwarnings('ignore')

In [245]:
import pandas_datareader as pdr
import datetime 
data = pdr.get_data_yahoo('AAPL', 
                          start=datetime.datetime(2006, 10, 1), 
                          end=datetime.datetime(2016, 1, 1))

data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-10-02,10.728572,10.838572,10.614285,10.694285,9.586717,178159800
2006-10-03,10.635715,10.707143,10.455714,10.582857,9.486828,197677200
2006-10-04,10.585714,10.780000,10.451428,10.768572,9.653308,207270700
2006-10-05,10.647142,10.880000,10.590000,10.690000,9.582876,170970800
2006-10-06,10.631429,10.720000,10.544286,10.602858,9.504756,116739700


In [86]:
ta_inputs = {
    'open': stock.Open,
    'high': stock.High,
    'low': stock.Low,
    'close': stock['Adj Close'], #adjusted close is used
    'volume': stock.Volume
}

In [246]:
def sma_signal(inputs, n):
    """SMA divided by adjusted close"""
    signal = pd.Series(ta.SMA(inputs, n)/inputs['close'], name='SMA_signal')
    return signal

def ema_signal(inputs, smooth_param):
    """EMA divided by adjusted close"""
    signal = pd.Series(ta.EMA(inputs, 2.0/smooth_param - 1), name='EMA_signal')
    return signal

def bb_signal(inputs, n, s_upper, s_lower):
    """signal 1 and 2: adjusted close respectively divided by upperband and lowerband;
    signal 3: 
    _Buy (return 1) if previous close >= lowerband and close < upperband
    _Sell (return -1) if previous close <= lowerband and close > upperband
    _Hold (return 0) else"""
    upperband, middleband, lowerband = ta.BBANDS(inputs, n, s_upper, s_lower)
    signal_upper = pd.Series(inputs['close'] / upperband, name = 'BB_upper')
    signal_lower = pd.Series(inputs['close'] / lowerband, name = 'BB_lower')
    
    today_close = list(inputs['close'])
    prev_close = ([np.nan] + today_close)[:-1]
    buy_cond = (prev_close >= lowerband) & (close < upperband)
    sell_cond = (prev_close <= lowerband) & (close > upperband)
    signal_trading = pd.Series(np.where(buy_cond, 1, np.where(sell_cond, -1, 0)), index = signal_upper.index,\
                               name = 'BB_trading')

    return signal_upper, signal_lower, signal_trading

In [6]:
# data = pd.concat([SMA_vars, EMA_vars, BB_vars], axis=1)
# pos_output_cond = stock['Adj Close'].pct_change() >= 0
# data['output'] = np.where(pos_output_cond, 1, 0)
# data.dropna(inplace=True)
# X = data.loc[:, data.columns != 'output'].as_matrix()
# y = data.loc[:, data.columns == 'output'].as_matrix()
# total_size = X.shape[0]
# train_size = int(total_size*0.5)
# val_size = int(total_size*0.1)
# X_train = X[:train_size,:]
# X_val = X[train_size + 1:val_size, :]
# X_test = X[train_size + val_size + 1:, :]
# y_train = y[:train_size,:]
# y_val = y[train_size + 1:val_size,:]
# y_test = y[train_size + val_size + 1:, :]